# 파편화 완료!

In [ ]:
import pygame
import random

pygame.font.init()

s_width = 800
s_height = 700
play_width = 300 
play_height = 600 
block_size = 30
top_left_x = (s_width - play_width) // 2
top_left_y = s_height - play_height

S = [['.....','.....','..00.','.00..','.....'],
     ['.....','..0..','..00.','...0.','.....']]

Z = [['.....','.....','.00..','..00.','.....'],
     ['.....','..0..','.00..','.0...','.....']]

I = [['..0..','..0..','..0..', '..0..','.....'],
     ['.....','0000.','.....','.....','.....']]

O = [['.....','.....','.00..','.00..','.....']]

J = [['.....','.0...','.000.','.....','.....'],
     ['.....','..00.','..0..','..0..','.....'],
     ['.....','.....','.000.','...0.','.....'],
     ['.....','..0..','..0..','.00..', '.....']]

L = [['.....','...0.','.000.','.....','.....'],
     ['.....','..0..','..0..','..00.','.....'],
     ['.....','.....','.000.','.0...', '.....'],
     ['.....','.00..','..0..','..0..', '.....']]

T = [['.....','..0..','.000.','.....','.....'],
     ['.....','..0..','..00.','..0..','.....'],
     ['.....','.....','.000.','..0..','.....'],
     ['.....','..0..','.00..','..0..','.....']]

P = [['.....','..0..','.000.','.....','.....'],
     ['.....','.....','.000.','..0..','.....']]

shapes = [S, Z, I, O, J, L, T, P]
shape_colors = [(255, 60, 60), (255, 150, 60), (255, 255, 50), (160, 255, 128)]

# 이미지만 넣어주면 됨

class Piece(object):
    rows = 20  
    columns = 10  

    def __init__(self, column, row, shape):
        self.x = column
        self.y = row
        self.shape = shape
        self.color = shape_colors[random.randint(0,3)]
        self.rotation = 0  


def create_grid(locked_positions={}):
    grid = [[(255, 255, 255) for x in range(10)] for x in range(20)]

    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (j, i) in locked_positions:
                c = locked_positions[(j, i)]
                grid[i][j] = c
    return grid


def convert_shape_format(shape):
    positions = []
    format = shape.shape[shape.rotation % len(shape.shape)]

    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                positions.append((shape.x + j, shape.y + i))

    for i, pos in enumerate(positions):
        positions[i] = (pos[0] - 2, pos[1] - 4)

    return positions


def valid_space(shape, grid):
    accepted_positions = [[(j, i) for j in range(10) if grid[i][j] == (255, 255, 255)] for i in range(20)]
    accepted_positions = [j for sub in accepted_positions for j in sub]
    formatted = convert_shape_format(shape)

    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False

    return True


def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False


def get_shape():
    global shapes, shape_colors
    return Piece(5, 0, random.choice(shapes))


def draw_text_middle(text, size, color, surface):
    font = pygame.font.SysFont('comicsans', size, bold=True)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), top_left_y + play_height / 2 - label.get_height() / 2))


def draw_main_menu(size,color,surface):
    font = pygame.font.SysFont('comicsans', size, bold=False)
    font_title = pygame.font.SysFont('comicsans', 2*size, bold=True)
    
    title = "T E T R I S"
    text1 = "Press Space To Play"
    text2 = "Press Esc to Quit"
    label1 = font.render(text1,1,color)
    label2 = font.render(text2,1,color)
    label_title = font_title.render(title,1,color)
    
    x1 = s_width / 2 - label1.get_width() / 2
    x2 = s_width / 2 - label2.get_width() / 2
    xt = s_width / 2 - label_title.get_width() / 2
    y1 = s_height /2 - label1.get_height() + label_title.get_height()
    y2 = s_height /2 - label2.get_height() + label1.get_height() + label_title.get_height()+ 10
    yt = s_height /2 - label_title.get_height()
    surface.blit(label_title, (xt,yt))
    
    surface.blit(label1, (x1,y1))
    surface.blit(label2, (x2, y2))


def draw_grid(surface, row, col):
    sx = top_left_x
    sy = top_left_y
    for i in range(row):
        pygame.draw.line(surface, (128, 128, 128), (sx, sy + i * 30),
                         (sx + play_width, sy + i * 30))  
        for j in range(col):
            pygame.draw.line(surface, (128, 128, 128), (sx + j * 30, sy),
                             (sx + j * 30, sy + play_height))


def add_fragmentation_effect(shape_pos):
    fragmented_pos = []
    for (x, y) in shape_pos:
        if random.random() > 0.5:  # 50% 확률로 위치를 변경
            # 파편화된 위치를 약간 변경
            fragmented_pos.append((x + random.choice([-1, 1]), y + random.choice([-1, 1])))
        else:
            fragmented_pos.append((x, y))
    return fragmented_pos
         

def clear_rows(grid, locked,score):
    inc = 0
    for i in range(len(grid) - 1, -1, -1):
        row = grid[i]
        #Clear if there is no white pixels in the row
        if (255, 255, 255) not in row:
            inc += 1
            ind = i
            score[0] = score[0] + 10;
            print(score)
            for j in range(len(row)):
                try:
                    del locked[(j, i)]
                except:
                    continue               
    if inc > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)



def draw_right_side(shape, surface,score):
    #Preview Next Shape
    font = pygame.font.SysFont('comicsans', 30)
    
    label = font.render('Next Shape', 1, (255, 255, 255))
    
    sx = top_left_x + play_width 
    sy = s_height / 2
    l  = (s_width - play_width) / 2;
    x = sx + l/2 
    x_line = sx + l/2 -2.5*block_size
    format = shape.shape[shape.rotation % len(shape.shape)]
    
    for i, line in enumerate(format):
        row = list(line)  
        for j, column in enumerate(row):
            if column == '0':
                pygame.draw.rect(surface, shape.color, (x_line + j * block_size, sy + i * block_size, block_size, block_size), 0)
    for i in range(6):
        pygame.draw.line(surface, (64, 64, 64), (x_line, sy + i * block_size),
                         (x_line + 6*block_size, sy + i * block_size))
        for j in range(6):
            pygame.draw.line(surface, (64, 64, 64), (x_line + j * block_size, sy),
                             (x_line + j * block_size, sy + 6*block_size))
    surface.blit(label, (x- label.get_width()/2, sy - block_size))
    
    #Preview Score
    label1 = font.render("SCORE", 1, (255, 255, 255))
    label2 = font.render(score, 1, (255, 255, 255))
    
    surface.blit(label1, (x- label1.get_width()/2, sy + 5*block_size))
    surface.blit(label2, (x- label2.get_width()/2, sy + 6*block_size))


def draw_score(surface,score) :
    sx = top_left_x + play_width + 50
    sy = top_left_y + play_height / 2 - 100
    font = pygame.font.SysFont('comicsans', 30)
    label1 = font.render("SCORE", 1, (255, 255, 255))
    label2 = font.render(str(score[0]), 1, (255, 255, 255))
    surface.blit(label1, (sx + 35, sy + 150))
    surface.blit(label2, (sx + 70, sy + 180))


def draw_window(surface):
    surface.fill((64, 64, 64))
    font = pygame.font.SysFont('comicsans', 60)
    label = font.render('T E T R I S', 1, (255, 255, 255))
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), 30))
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            pygame.draw.rect(surface, grid[i][j], (top_left_x + j * 30, top_left_y + i * 30, 30, 30), 0)      
    draw_grid(surface, 20, 10)
    pygame.draw.rect(surface, (100, 100, 100), (top_left_x, top_left_y, play_width, play_height), 5)



def main():
    global grid
    Score = [0]
    locked_positions = {}  
    grid = create_grid(locked_positions)

    change_piece = False
    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    clock = pygame.time.Clock()
    fall_time = 0

    while run:
        fall_speed = 0.27

        grid = create_grid(locked_positions)
        fall_time += clock.get_rawtime()
        clock.tick()

        if fall_time / 1000 >= fall_speed:
            fall_time = 0
            current_piece.y += 1
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:
                current_piece.y -= 1
                change_piece = True

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()
                quit()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_piece.x -= 1
                    if not valid_space(current_piece, grid):
                        current_piece.x += 1

                elif event.key == pygame.K_RIGHT:
                    current_piece.x += 1
                    if not valid_space(current_piece, grid):
                        current_piece.x -= 1
                elif event.key == pygame.K_UP:
                    # rotate shape
                    current_piece.rotation = current_piece.rotation + 1 % len(current_piece.shape)
                    if not valid_space(current_piece, grid):
                        current_piece.rotation = current_piece.rotation - 1 % len(current_piece.shape)

                if event.key == pygame.K_DOWN:
                    # move shape down
                    current_piece.y += 1
                    if not valid_space(current_piece, grid):
                        current_piece.y -= 1

        shape_pos = convert_shape_format(current_piece)

   
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.color

        shape_pos = convert_shape_format(current_piece)

        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.color

        if change_piece:
            # 파편화 효과 적용
            shape_pos = add_fragmentation_effect(shape_pos)
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.color
            current_piece = next_piece
            next_piece = get_shape()
            change_piece = False

    
            clear_rows(grid, locked_positions,Score)

        draw_window(win)
        score = str(Score[0])
        draw_right_side(next_piece, win,score)
        pygame.display.update()

        if check_lost(locked_positions):
            run = False
            
    draw_text_middle("You Lost! Score : " + str(Score[0]), 40, (0, 0, 0), win)
    pygame.display.update()
    pygame.time.delay(2000)

def main_menu():
    clk = 0;
    run = True
    color = (0,0,0)
    while run:
        if clk % 501 == 500 :
            color = random.choice(shape_colors)
        win.fill(color)
        draw_main_menu(60,(255,255,255),win)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE :
                    main()
                elif  event.key == pygame.K_ESCAPE :
                    run = False
        clk+=1            
    pygame.quit()




win = pygame.display.set_mode((s_width, s_height))
pygame.display.set_caption('T E T R I S')

main_menu()  



# 이미지 넣자!

In [ ]:
import pygame
import random

pygame.font.init()

s_width = 800
s_height = 700
play_width = 300 
play_height = 600 
block_size = 30
row = play_height // block_size
column = play_width // block_size
top_left_x = (s_width - play_width) // 2
top_left_y = s_height - play_height

S = [['.....','.....','..00.','.00..','.....'],
     ['.....','..0..','..00.','...0.','.....']]

Z = [['.....','.....','.00..','..00.','.....'],
     ['.....','..0..','.00..','.0...','.....']]

I = [['..0..','..0..','..0..', '..0..','.....'],
     ['.....','0000.','.....','.....','.....']]

O = [['.....','.....','.00..','.00..','.....']]

J = [['.....','.0...','.000.','.....','.....'],
     ['.....','..00.','..0..','..0..','.....'],
     ['.....','.....','.000.','...0.','.....'],
     ['.....','..0..','..0..','.00..', '.....']]

L = [['.....','...0.','.000.','.....','.....'],
     ['.....','..0..','..0..','..00.','.....'],
     ['.....','.....','.000.','.0...', '.....'],
     ['.....','.00..','..0..','..0..', '.....']]

T = [['.....','..0..','.000.','.....','.....'],
     ['.....','..0..','..00.','..0..','.....'],
     ['.....','.....','.000.','..0..','.....'],
     ['.....','..0..','.00..','..0..','.....']]

P = [['.....','..0..','.000.','.....','.....'],
     ['.....','.....','.000.','..0..','.....']]

shapes = [S, Z, I, O, J, L, T, P]
shape_colors = [(255, 60, 60), (255, 150, 60), (255, 255, 50), (160, 255, 128)]


fire = pygame.image.load('img/fire.png')  # 예시 이미지 파일명
fire = pygame.transform.scale(fire, (block_size, block_size))  # 조각 크기에 맞게 조절

glass = pygame.image.load('img/glass.png')  # 예시 이미지 파일명
glass = pygame.transform.scale(glass, (block_size, block_size))  # 조각 크기에 맞게 조절

stone = pygame.image.load('img/stone.png')  # 예시 이미지 파일명
stone = pygame.transform.scale(stone, (block_size, block_size))  # 조각 크기에 맞게 조절

tree = pygame.image.load('img/tree.png')  # 예시 이미지 파일명
tree = pygame.transform.scale(tree, (block_size, block_size))  # 조각 크기에 맞게 조절

images = [fire,glass,stone,tree]

class Piece(object):
    def __init__(self, column, row, shape):
        self.x = column
        self.y = row
        self.shape = shape
        self.color = shape_colors[random.randint(0,3)]
        self.image = random.choice(images) # 수정사항
        self.rotation = 0  


def create_grid(locked_positions={}): # 배경생성
    grid = [[(255, 255, 255) for x in range(column)] for x in range(row)] # 10곱하기 20 사이즈

    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (j, i) in locked_positions:
                c = locked_positions[(j, i)]
                grid[i][j] = c
    return grid


# 파편화
def convert_shape_format(shape):
    positions = []
    format = shape.shape[shape.rotation % len(shape.shape)]

    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                positions.append((shape.x + j, shape.y + i))

    for i, pos in enumerate(positions):
        if pos[0] < 0:  # 좌측 경계를 벗어나는 경우
            shift = abs(pos[0])
            for j in range(len(positions)):
                positions[j] = (positions[j][0] + shift, positions[j][1])
            shape.x += shift
        elif pos[0] >= 10:  # 우측 경계를 벗어나는 경우
            shift = 10 - pos[0]
            for j in range(len(positions)):
                positions[j] = (positions[j][0] + shift, positions[j][1])
            shape.x += shift
        if pos[1] >= 20:  # 하단 경계를 벗어나는 경우
            return positions  # 바닥에 닿았을 때는 위치 조정하지 않고 현재 위치 그대로 반환합니다.

    return positions


def valid_space(shape, grid):
    accepted_positions = [[(j, i) for j in range(10) if grid[i][j] == (255, 255, 255)] for i in range(20)]
    accepted_positions = [j for sub in accepted_positions for j in sub]
    formatted = convert_shape_format(shape)

    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False

    return True


def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False



def get_shape():
    global shapes, shape_colors
    shape = Piece(column // 2 - 1, 0, random.choice(shapes))  # 그리드의 중앙에 위치하도록 수정
    return shape



def draw_text_middle(text, size, color, surface):
    font = pygame.font.SysFont('comicsans', size, bold=True)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), top_left_y + play_height / 2 - label.get_height() / 2))


def draw_main_menu(size,color,surface):
    font = pygame.font.SysFont('comicsans', size, bold=False)
    font_title = pygame.font.SysFont('comicsans', 2*size, bold=True)
    
    title = "T E T R I S"
    text1 = "Press Space To Play"
    text2 = "Press Esc to Quit"
    label1 = font.render(text1,1,color)
    label2 = font.render(text2,1,color)
    label_title = font_title.render(title,1,color)
    
    x1 = s_width / 2 - label1.get_width() / 2
    x2 = s_width / 2 - label2.get_width() / 2
    xt = s_width / 2 - label_title.get_width() / 2
    y1 = s_height /2 - label1.get_height() + label_title.get_height()
    y2 = s_height /2 - label2.get_height() + label1.get_height() + label_title.get_height()+ 10
    yt = s_height /2 - label_title.get_height()
    surface.blit(label_title, (xt,yt))
    
    surface.blit(label1, (x1,y1))
    surface.blit(label2, (x2, y2))


def draw_grid(surface, row, col):
    sx = top_left_x
    sy = top_left_y
    for i in range(row):
        pygame.draw.line(surface, (128, 128, 128), (sx, sy + i * block_size),
                         (sx + play_width, sy + i * block_size))  
        for j in range(col):
            pygame.draw.line(surface, (128, 128, 128), (sx + j * block_size, sy),
                             (sx + j * block_size, sy + play_height))


def add_fragmentation_effect(shape_pos):
    fragmented_pos = []
    if shape_pos:  # 테트로미노의 위치가 비어 있지 않은 경우에만 작동
        for (x, y) in shape_pos:
            if y < 19 and (x, y + 1) not in shape_pos and random.random() < 0.3:  # 테트로미노가 아래에 있지 않고 맨 아래에 있는 조각일 때만 파편화 효과를 받습니다.
                # 30% 확률로 위치를 변경
                # 파편화된 위치를 약간 변경
                new_x = x + random.choice([-1, 1])
                new_y = y + random.choice([-1, 1])
                # 새로운 위치가 그리드를 벗어나지 않도록 조정
                new_x = max(0, min(new_x, 9))  # x 좌표를 0과 9 사이로 조정
                new_y = max(0, min(new_y, 19))  # y 좌표를 0과 19 사이로 조정
                fragmented_pos.append((new_x, new_y))
            else:
                fragmented_pos.append((x, y))
    else:
        fragmented_pos = shape_pos  # 테트로미노의 위치가 없으면 그대로 반환
    return fragmented_pos
# def add_fragmentation_effect(shape_pos):
#     fragmented_pos = []
#     if shape_pos:  # 테트로미노의 위치가 비어 있지 않은 경우에만 작동
#         for (x, y) in shape_pos:
#             if (x, y + 1) not in shape_pos and random.random() < 0.3:  # 아래에 블록이 없고 30% 확률로 파편화 효과를 적용
#                 # 파편화된 위치를 약간 변경
#                 new_x = x + random.choice([-1, 1])
#                 new_y = y + random.choice([-1, 1])
#                 # 새로운 위치가 그리드를 벗어나지 않도록 조정
#                 new_x = max(0, min(new_x, 10))  # x 좌표를 0과 9 사이로 조정
#                 new_y = max(0, min(new_y, 20))  # y 좌표를 0과 19 사이로 조정
#                 fragmented_pos.append((new_x, new_y))
#             else:
#                 fragmented_pos.append((x, y))
#     else:
#         fragmented_pos = shape_pos  # 테트로미노의 위치가 없으면 그대로 반환
#     return fragmented_pos


         

def clear_rows(grid, locked,score):
    inc = 0
    for i in range(len(grid) - 1, -1, -1):
        row = grid[i]
        #Clear if there is no white pixels in the row
        if (255, 255, 255) not in row:
            inc += 1
            ind = i
            score[0] = score[0] + 10;
            print(score)
            for j in range(len(row)):
                try:
                    del locked[(j, i)]
                except:
                    continue               
    if inc > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)



# def draw_right_side(shape, surface, score):
#     # Preview Next Shape
#     font = pygame.font.SysFont('comicsans', 30)
    
#     label_next = font.render('Next Shape', 1, (255, 255, 255))
    
#     sx = top_left_x + play_width + 50
#     sy = top_left_y + (play_height / 2 - 100)
#     format = shape.shape[shape.rotation % len(shape.shape)]
    
#     for i, line in enumerate(format):
#         row = list(line)
#         for j, column in enumerate(row):
#             if column == '0':
#                 surface.blit(shape.image, (sx + j * block_size, sy + i * block_size))
    
#     # Draw Score
#     sx_score = top_left_x + play_width + 50
#     sy_score = top_left_y + (play_height / 2 - 100) + 250
    
#     label_score = font.render('Score:', 1, (255, 255, 255))
#     label_value = font.render(str(score[0]), 1, (255, 255, 255))
    
#     surface.blit(label_next, (sx, sy - 50))
#     surface.blit(label_score, (sx_score, sy_score))
#     surface.blit(label_value, (sx_score, sy_score + 40))

def draw_right_side(next_piece, surface,score):
    # Preview Next Shape
    font = pygame.font.SysFont('comicsans', 30)
    
    label_next = font.render('Next Shape', 1, (255, 255, 255))
    
    sx = top_left_x + play_width + 50
    sy = top_left_y + (play_height / 2 - 100)
    format = next_piece.shape[next_piece.rotation % len(next_piece.shape)]
    
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                surface.blit(next_piece.image, (sx + j * block_size, sy + i * block_size))
    
    # Draw Score
    sx_score = top_left_x + play_width + 50
    sy_score = top_left_y + (play_height / 2 - 100) + 250
    
    label_score = font.render('Score:', 1, (255, 255, 255))
    label_value = font.render(str(score), 1, (255, 255, 255))
    
    surface.blit(label_next, (sx, sy - 50))
    surface.blit(label_score, (sx_score, sy_score))
    surface.blit(label_value, (sx_score, sy_score + 40))
    pygame.display.update()



def draw_score(surface,score) :
    sx = top_left_x + play_width + 50
    sy = top_left_y + play_height / 2 - 100
    font = pygame.font.SysFont('comicsans', 30)
    label1 = font.render("SCORE", 1, (255, 255, 255))
    label2 = font.render(str(score[0]), 1, (255, 255, 255))
    surface.blit(label1, (sx + 35, sy + 150))
    surface.blit(label2, (sx + 70, sy + 180))


# 테트로미노 조각에 대한 이미지

def draw_shape(surface, piece, offset_x, offset_y):
    shape_pos = convert_shape_format(piece)
    for x, y in shape_pos:
        if y > -1:
            surface.blit(piece.image, (offset_x + x * block_size, offset_y + y * block_size))


def draw_window(surface, current_piece, locked_positions):
    surface.fill((64, 64, 64))
    font = pygame.font.SysFont('comicsans', 60)
    label = font.render('T E T R I S', 1, (255, 255, 255))
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), 30))
    
    # 현재 조각 그리기
    draw_shape(surface, current_piece, top_left_x, top_left_y)
    
    # 고정된 조각 그리기
    for pos in locked_positions:
        x, y = pos
        image = locked_positions[pos]
        surface.blit(image, (top_left_x + x * block_size, top_left_y + y * block_size))
    
    draw_grid(surface, 20, 10)
    pygame.draw.rect(surface, (100, 100, 100), (top_left_x, top_left_y, play_width, play_height), 5)
    pygame.display.update()




def main():
    global grid
    Score = [0]
    locked_positions = {}  
    grid = create_grid(locked_positions)

    change_piece = False
    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    clock = pygame.time.Clock()
    fall_time = 0

    while run:
        fall_speed = 0.2

        grid = create_grid(locked_positions)
        fall_time += clock.get_rawtime()
        clock.tick()

        if fall_time / 1000 >= fall_speed:
            fall_time = 0
            current_piece.y += 1
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:
                current_piece.y -= 1
                change_piece = True

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()
                quit()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_piece.x -= 1
                    if not valid_space(current_piece, grid):
                        current_piece.x += 1

                elif event.key == pygame.K_RIGHT:
                    current_piece.x += 1
                    if not valid_space(current_piece, grid):
                        current_piece.x -= 1
                elif event.key == pygame.K_UP:
                    # rotate shape
                    current_piece.rotation = current_piece.rotation + 1 % len(current_piece.shape)
                    if not valid_space(current_piece, grid):
                        current_piece.rotation = current_piece.rotation - 1 % len(current_piece.shape)

                if event.key == pygame.K_DOWN:
                    # move shape down
                    current_piece.y += 1
                    if not valid_space(current_piece, grid):
                        current_piece.y -= 1

        shape_pos = convert_shape_format(current_piece)

   
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.image


        if change_piece:
            # 파편화 효과 적용
            shape_pos = add_fragmentation_effect(shape_pos)
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.image
            current_piece = next_piece
            next_piece = get_shape()
            change_piece = False

    
            clear_rows(grid, locked_positions,Score)

        draw_window(win,current_piece,locked_positions)
        score = str(Score[0])
        draw_right_side(next_piece, win,score)
        # pygame.display.update()

        if check_lost(locked_positions):
            run = False
            
    draw_text_middle("You Lost! Score : " + str(Score[0]), 40, (0, 0, 0), win)
    pygame.display.update()
    pygame.time.delay(2000)

def main_menu():
    clk = 0;
    run = True
    color = (0,0,0)
    while run:
        if clk % 501 == 500 :
            color = random.choice(shape_colors)
        win.fill(color)
        draw_main_menu(60,(255,255,255),win)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE :
                    main()
                elif  event.key == pygame.K_ESCAPE :
                    run = False
        clk+=1            
    pygame.quit()




win = pygame.display.set_mode((s_width, s_height))
pygame.display.set_caption('T E T R I S')

main_menu()  



# 깜빡이는 거 해결하면 됨
# 그리고 조건 추가, 지금은 그냥 만나면 작동함

In [9]:
import pygame
import random

pygame.font.init()

s_width = 800
s_height = 700
play_width = 300 
play_height = 600 
block_size = 30
row = play_height // block_size
column = play_width // block_size
top_left_x = (s_width - play_width) // 2
top_left_y = s_height - play_height

S = [['.....','.....','..00.','.00..','.....'],
     ['.....','..0..','..00.','...0.','.....']]

Z = [['.....','.....','.00..','..00.','.....'],
     ['.....','..0..','.00..','.0...','.....']]

I = [['..0..','..0..','..0..', '..0..','.....'],
     ['.....','0000.','.....','.....','.....']]

O = [['.....','.....','.00..','.00..','.....']]

J = [['.....','.0...','.000.','.....','.....'],
     ['.....','..00.','..0..','..0..','.....'],
     ['.....','.....','.000.','...0.','.....'],
     ['.....','..0..','..0..','.00..', '.....']]

L = [['.....','...0.','.000.','.....','.....'],
     ['.....','..0..','..0..','..00.','.....'],
     ['.....','.....','.000.','.0...', '.....'],
     ['.....','.00..','..0..','..0..', '.....']]

T = [['.....','..0..','.000.','.....','.....'],
     ['.....','..0..','..00.','..0..','.....'],
     ['.....','.....','.000.','..0..','.....'],
     ['.....','..0..','.00..','..0..','.....']]

P = [['.....','..0..','.000.','.....','.....'],
     ['.....','.....','.000.','..0..','.....'],
     ['.....','.....','.000.','..0..','.....'],
     ['.....','.....','.000.','..0..','.....']]

T = [['.....','..0..','.000.','.....','.....'],
     ['..0..','..00.','..0..','.....','.....'],
     ['.....','.000.','..0..','.....','.....'],
     ['..0..','.00..','..0..','.....','.....']]


shapes = [S, Z, I, O, J, L, T, P]
shape_colors = [(255, 60, 60), (255, 150, 60), (255, 255, 50), (160, 255, 128)]


fire = pygame.image.load('img/fire.png')  # 예시 이미지 파일명
fire = pygame.transform.scale(fire, (block_size, block_size))  # 조각 크기에 맞게 조절

glass = pygame.image.load('img/glass.png')  # 예시 이미지 파일명
glass = pygame.transform.scale(glass, (block_size, block_size))  # 조각 크기에 맞게 조절

stone = pygame.image.load('img/stone.png')  # 예시 이미지 파일명
stone = pygame.transform.scale(stone, (block_size, block_size))  # 조각 크기에 맞게 조절

tree = pygame.image.load('img/tree.png')  # 예시 이미지 파일명
tree = pygame.transform.scale(tree, (block_size, block_size))  # 조각 크기에 맞게 조절

images = [fire,glass,stone,tree]

class Piece(object):
    def __init__(self, column, row, shape):
        self.x = column
        self.y = row
        self.shape = shape
        self.color = shape_colors[random.randint(0,3)]
        self.image = random.choice(images) # 수정사항
        self.rotation = 0  


def create_grid(locked_positions={}): # 배경생성
    grid = [[(255, 255, 255) for x in range(column)] for x in range(row)] # 10곱하기 20 사이즈

    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (j, i) in locked_positions:
                c = locked_positions[(j, i)]
                grid[i][j] = c
    return grid


def convert_shape_format(shape):
    positions = []
    format = shape.shape[shape.rotation % len(shape.shape)]

    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                positions.append((shape.x + j, shape.y + i))

    for i, pos in enumerate(positions):
        if pos[0] < 0:  # 좌측 경계를 벗어나는 경우
            shift = abs(pos[0])
            for j in range(len(positions)):
                positions[j] = (positions[j][0] + shift, positions[j][1])
            shape.x += shift
        elif pos[0] >= 10:  # 우측 경계를 벗어나는 경우
            shift = 10 - pos[0]
            for j in range(len(positions)):
                positions[j] = (positions[j][0] + shift, positions[j][1])
            shape.x += shift
        if pos[1] >= 20:  # 하단 경계를 벗어나는 경우
            return positions  # 바닥에 닿았을 때는 현재 위치 그대로 반환합니다.

    return positions



def valid_space(shape, grid):
    accepted_positions = [[(j, i) for j in range(10) if grid[i][j] == (255, 255, 255)] for i in range(20)]
    accepted_positions = [j for sub in accepted_positions for j in sub]
    formatted = convert_shape_format(shape)

    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False

    return True


def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False



def get_shape():
    global shapes, shape_colors
    shape = Piece(column // 2 - 2, -2, random.choice(shapes))  # 그리드의 중앙에 위치하도록 수정
    return shape



def draw_text_middle(text, size, color, surface):
    font = pygame.font.SysFont('comicsans', size, bold=True)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), top_left_y + play_height / 2 - label.get_height() / 2))


def draw_main_menu(size,color,surface):
    font = pygame.font.SysFont('comicsans', size, bold=False)
    font_title = pygame.font.SysFont('comicsans', 2*size, bold=True)
    
    title = "T E T R I S"
    text1 = "Press Space To Play"
    text2 = "Press Esc to Quit"
    label1 = font.render(text1,1,color)
    label2 = font.render(text2,1,color)
    label_title = font_title.render(title,1,color)
    
    x1 = s_width / 2 - label1.get_width() / 2
    x2 = s_width / 2 - label2.get_width() / 2
    xt = s_width / 2 - label_title.get_width() / 2
    y1 = s_height /2 - label1.get_height() + label_title.get_height()
    y2 = s_height /2 - label2.get_height() + label1.get_height() + label_title.get_height()+ 10
    yt = s_height /2 - label_title.get_height()
    surface.blit(label_title, (xt,yt))
    
    surface.blit(label1, (x1,y1))
    surface.blit(label2, (x2, y2))


def draw_grid(surface, row, col):
    sx = top_left_x
    sy = top_left_y
    for i in range(row):
        pygame.draw.line(surface, (128, 128, 128), (sx, sy + i * block_size),
                         (sx + play_width, sy + i * block_size))  
        for j in range(col):
            pygame.draw.line(surface, (128, 128, 128), (sx + j * block_size, sy),
                             (sx + j * block_size, sy + play_height))


def add_fragmentation_effect(shape_pos):
    fragmented_pos = []
    if shape_pos:  # 테트로미노의 위치가 비어 있지 않은 경우에만 작동
        for (x, y) in shape_pos:
            if y < 19 and (x, y + 1) not in shape_pos and random.random() < 0.3:  # 테트로미노가 아래에 있지 않고 맨 아래에 있는 조각일 때만 파편화 효과를 받습니다.
                # 30% 확률로 위치를 변경
                # 파편화된 위치를 약간 변경
                new_x = x + random.choice([-1, 1])
                new_y = y + random.choice([-1, 1])
                # 새로운 위치가 그리드를 벗어나지 않도록 조정
                new_x = max(0, min(new_x, 9))  # x 좌표를 0과 9 사이로 조정
                new_y = max(0, min(new_y, 19))  # y 좌표를 0과 19 사이로 조정
                fragmented_pos.append((new_x, new_y))
            else:
                fragmented_pos.append((x, y))
    else:
        fragmented_pos = shape_pos  # 테트로미노의 위치가 없으면 그대로 반환
    return fragmented_pos



         

def clear_rows(grid, locked,score):
    inc = 0
    for i in range(len(grid) - 1, -1, -1):
        row = grid[i]
        #Clear if there is no white pixels in the row
        if (255, 255, 255) not in row:
            inc += 1
            ind = i
            score[0] = score[0] + 10;
            print(score)
            for j in range(len(row)):
                try:
                    del locked[(j, i)]
                except:
                    continue               
    if inc > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)



def draw_right_side(next_piece, surface,score):
    # Preview Next Shape
    font = pygame.font.SysFont('comicsans', 30)
    
    label_next = font.render('Next Shape', 1, (255, 255, 255))
    
    sx = top_left_x + play_width + 50
    sy = top_left_y + (play_height / 2 - 100)
    format = next_piece.shape[next_piece.rotation % len(next_piece.shape)]
    
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                surface.blit(next_piece.image, (sx + j * block_size, sy + i * block_size))
    
    # Draw Score
    sx_score = top_left_x + play_width + 50
    sy_score = top_left_y + (play_height / 2 - 100) + 250
    
    label_score = font.render('Score:', 1, (255, 255, 255))
    label_value = font.render(str(score), 1, (255, 255, 255))
    
    surface.blit(label_next, (sx, sy - 50))
    surface.blit(label_score, (sx_score, sy_score))
    surface.blit(label_value, (sx_score, sy_score + 40))
    pygame.display.update()



def draw_score(surface,score) :
    sx = top_left_x + play_width + 50
    sy = top_left_y + play_height / 2 - 100
    font = pygame.font.SysFont('comicsans', 30)
    label1 = font.render("SCORE", 1, (255, 255, 255))
    label2 = font.render(str(score[0]), 1, (255, 255, 255))
    surface.blit(label1, (sx + 35, sy + 150))
    surface.blit(label2, (sx + 70, sy + 180))


# 테트로미노 조각에 대한 이미지

def draw_shape(surface, piece, offset_x, offset_y):
    shape_pos = convert_shape_format(piece)
    for x, y in shape_pos:
        if y > -1:
            surface.blit(piece.image, (offset_x + x * block_size, offset_y + y * block_size))


def draw_window(surface, current_piece, locked_positions):
    surface.fill((64, 64, 64))
    font = pygame.font.SysFont('comicsans', 60)
    label = font.render('T E T R I S', 1, (255, 255, 255))
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), 30))
    
    # 현재 조각 그리기
    draw_shape(surface, current_piece, top_left_x, top_left_y)
    
    # 고정된 조각 그리기
    for pos in locked_positions:
        x, y = pos
        image = locked_positions[pos]
        surface.blit(image, (top_left_x + x * block_size, top_left_y + y * block_size))
    
    draw_grid(surface, 20, 10)
    pygame.draw.rect(surface, (100, 100, 100), (top_left_x, top_left_y, play_width, play_height), 5)
    pygame.display.update()




def main():
    global grid
    Score = [0]
    locked_positions = {}  
    grid = create_grid(locked_positions)

    change_piece = False
    run = True
    current_piece = get_shape()
    next_piece = get_shape()
    clock = pygame.time.Clock()
    fall_time = 0
    level = 1

    while run:
        fall_speed = 0.2

        grid = create_grid(locked_positions)
        fall_time += clock.get_rawtime()
        clock.tick()

        if fall_time > level * 5:
            level += 1
            fall_speed -= level * 0.2

        if fall_time / 1000 >= fall_speed:
            fall_time = 0
            current_piece.y += 1
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:
                current_piece.y -= 1
                change_piece = True
        

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()
                quit()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_piece.x -= 1
                    if not valid_space(current_piece, grid):
                        current_piece.x += 1

                elif event.key == pygame.K_RIGHT:
                    current_piece.x += 1
                    if not valid_space(current_piece, grid):
                        current_piece.x -= 1
                elif event.key == pygame.K_UP:
                    # rotate shape
                    current_piece.rotation = current_piece.rotation + 1 % len(current_piece.shape)
                    if not valid_space(current_piece, grid):
                        current_piece.rotation = current_piece.rotation - 1 % len(current_piece.shape)

                if event.key == pygame.K_DOWN:
                    # move shape down
                    current_piece.y += 1
                    if not valid_space(current_piece, grid):
                        current_piece.y -= 1

        shape_pos = convert_shape_format(current_piece)

   
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_piece.image


        if change_piece:
            # 파편화 효과 적용
            shape_pos = add_fragmentation_effect(shape_pos)
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.image
            current_piece = next_piece
            next_piece = get_shape()
            change_piece = False

    
            clear_rows(grid, locked_positions,Score)

        draw_window(win,current_piece,locked_positions)
        score = str(Score[0])
        draw_right_side(next_piece, win,score)
        # pygame.display.update()

        if check_lost(locked_positions):
            run = False
            
    draw_text_middle("You Lost! Score : " + str(Score[0]), 40, (0, 0, 0), win)
    pygame.display.update()
    pygame.time.delay(2000)

def main_menu():
    clk = 0;
    run = True
    color = (0,0,0)
    while run:
        if clk % 501 == 500 :
            color = random.choice(shape_colors)
        win.fill(color)
        draw_main_menu(60,(255,255,255),win)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE :
                    main()
                elif  event.key == pygame.K_ESCAPE :
                    run = False
        clk+=1            
    pygame.quit()




win = pygame.display.set_mode((s_width, s_height))
pygame.display.set_caption('T E T R I S')

main_menu()  



[10]
[20]
[30]
[40]


In [8]:
pygame.display.quit()